In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import sklearn

In [2]:
rain_df = pd.read_csv('weatherAUS.csv')

In [3]:
rain_df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,02-12-2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,03-12-2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,04-12-2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,05-12-2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,21-06-2017,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,22-06-2017,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,23-06-2017,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,24-06-2017,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [4]:
#EDA
px.histogram(rain_df,
          x = 'Humidity9am',
          color = 'RainTomorrow',
          )

In [5]:
px.histogram (rain_df,
           x = 'MinTemp',
           color = 'RainTomorrow')

In [6]:
rain_df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [7]:
rain_df.dropna(subset=['RainTomorrow', 'RainToday'], inplace = True) 
#Dealing with missing values in target column

In [8]:
#Data Splitting.
#since this is a time-series data it is better to split train test and val data in a chronological manner
year = rain_df['Date'] = pd.to_datetime(rain_df.Date, dayfirst = True).dt.year #encountered a day first error

In [9]:
train_df = rain_df[year < 2015]
val_df = rain_df[year == 2015]
test_df = rain_df[year>2015]
#copy data

In [10]:
train_df.Date.count() #forgot how to check for no of rows so improvised 
val_df.Date.count()
test_df.Date.count()

np.int64(25710)

In [11]:
train_inputs = train_df.loc[:,'Location':'RainToday']
val_inputs = val_df.loc[:,'Location':'RainToday']
test_inputs = test_df.loc[:,'Location':'RainToday']

In [12]:
#defining categorical and numerical columns
cat_cols = list(train_inputs.select_dtypes(include = 'object'))
num_cols = list(train_inputs.select_dtypes(include = 'number'))

In [13]:
train_targets = train_df['RainTomorrow']
val_targets = val_df['RainTomorrow']
test_targets = test_df['RainTomorrow']

In [14]:
train_inputs

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144548,Uluru,16.9,33.2,0.0,NaN,NaN,SSE,43.0,ESE,SSE,...,26.0,22.0,13.0,1014.1,1009.8,NaN,NaN,23.7,31.8,No
144549,Uluru,15.1,36.8,0.0,NaN,NaN,NE,31.0,ENE,SW,...,20.0,16.0,8.0,1012.6,1007.6,NaN,NaN,28.9,34.8,No
144550,Uluru,17.3,37.8,0.0,NaN,NaN,ESE,39.0,ESE,SSE,...,9.0,15.0,8.0,1011.9,1008.0,NaN,NaN,29.7,35.7,No
144551,Uluru,20.1,38.5,0.0,NaN,NaN,ESE,43.0,ESE,SSW,...,17.0,22.0,9.0,1014.0,1009.2,NaN,NaN,29.8,37.2,No


In [15]:
# feature engineering numerical columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(rain_df[num_cols])

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [16]:
#Replacing Nan values with mean
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'mean')

In [17]:
train_num = pd.DataFrame()
train_num = pd.DataFrame()
test_num = pd.DataFrame()

In [18]:
train_num

""


In [19]:
train_num[num_cols]= scaler.transform(train_df[num_cols]) 

In [20]:
#Replacing Nan values with mean
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'mean')
imputer.fit(train_num)
train_inputs[num_cols]= imputer.transform(train_num) 

In [21]:
#OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
enc.fit(train_inputs[cat_cols])

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [24]:
enc_cols = enc.get_feature_names_out(cat_cols)

In [29]:
encoded_inputs

,Location_Adelaide,Location_Albany,Location_Albury,Location_AliceSprings,Location_BadgerysCreek,Location_Ballarat,Location_Bendigo,Location_Brisbane,Location_Cairns,Location_Canberra,...,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,WindDir3pm_nan,RainToday_No,RainToday_Yes
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
144549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
144550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
144551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
encoded_inputs = enc.transform(train_inputs[cat_cols])

In [27]:
encoded_inputs = pd.DataFrame(encoded_inputs, columns = enc_cols, index=train_inputs.index)

In [28]:
train_inputs = pd.concat([train_inputs.drop(cat_cols, axis=1), encoded_inputs], axis=1)

In [30]:
#model Training
from sklearn.linear_model import LogisticRegression

In [31]:
model = LogisticRegression(solver = 'liblinear').fit(train_inputs, train_targets)

In [46]:
train_inputs.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,WindDir3pm_nan,RainToday_No,RainToday_Yes
count,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,...,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000,97988.000000
mean,0.483675,0.525940,0.006396,0.036483,0.524759,0.265239,0.108402,0.215685,0.686287,0.514695,...,0.077591,0.064130,0.055323,0.064569,0.069815,0.061559,0.066825,0.019921,0.775626,0.224374
std,0.149457,0.131904,0.022962,0.021620,0.200930,0.102419,0.068800,0.101424,0.189008,0.206376,...,0.267528,0.244986,0.228611,0.245765,0.254836,0.240353,0.249719,0.139729,0.417172,0.417172
min,0.000000,0.013233,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.377358,0.429112,0.000000,0.026207,0.517241,0.193798,0.053846,0.149425,0.570000,0.370000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.478774,0.514178,0.000000,0.036483,0.524759,0.255814,0.100000,0.218391,0.690000,0.520000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.591981,0.618147,0.002156,0.038621,0.634483,0.310078,0.146154,0.275862,0.830000,0.650000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,0.568276,0.986207,1.000000,0.669231,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:
model.predict(va)

NameError: name 'va' is not defined

In [36]:
val_df.isna().sum()

Date                0
Location            0
MinTemp             0
MaxTemp             0
Rainfall            0
Evaporation         0
Sunshine            0
WindGustDir       893
WindGustSpeed       0
WindDir9am       1099
WindDir3pm        511
WindSpeed9am        0
WindSpeed3pm        0
Humidity9am         0
Humidity3pm         0
Pressure9am         0
Pressure3pm         0
Cloud9am            0
Cloud3pm            0
Temp9am             0
Temp3pm             0
RainToday           0
RainTomorrow        0
dtype: int64

In [35]:
#time to clean validation data. im goig to take the whole step again to make i tstick to my head.
from sklearn.impute import SimpleImputer
impute = SimpleImputer(strategy = 'mean')
impute.fit(val_df[num_cols])
val_df[num_cols] = impute.transform(val_df[num_cols])

C:\Users\pc\AppData\Local\Temp\ipykernel_2680\2406991249.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
scaler = MinMaxScaler()
scaler.fit(val_df[num_cols])

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [38]:
scaled_inputs = scaler.transform(val_df[num_cols])

In [39]:
val_inputs[num_cols] = scaled_inputs

In [40]:
val_inputs

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
2133,Albury,0.488778,0.755144,0.0,0.084307,0.534232,WSW,0.179688,ESE,W,...,0.148649,0.427083,0.14,0.496094,0.522686,0.559943,0.562783,0.622426,0.784188,No
2134,Albury,0.591022,0.880658,0.0,0.084307,0.534232,NE,0.382812,ESE,ESE,...,0.121622,0.427083,0.12,0.544922,0.548094,0.559943,0.562783,0.727689,0.901709,No
2135,Albury,0.630923,0.853909,0.0,0.084307,0.534232,NNE,0.320312,NE,N,...,0.270270,0.322917,0.19,0.582031,0.546279,0.559943,0.562783,0.810069,0.876068,No
2136,Albury,0.852868,0.746914,0.0,0.084307,0.534232,NNE,0.265625,ESE,W,...,0.094595,0.437500,0.37,0.498047,0.542650,1.000000,0.625000,0.768879,0.745726,No
2137,Albury,0.678304,0.790123,0.0,0.084307,0.534232,E,0.203125,SSE,SE,...,0.121622,0.583333,0.34,0.572266,0.589837,1.000000,0.562783,0.727689,0.779915,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144913,Uluru,0.715711,0.779835,0.0,0.084307,0.534232,E,0.351562,ESE,E,...,0.270270,0.197917,0.12,0.490234,0.506352,0.559943,0.562783,0.697941,0.790598,No
144914,Uluru,0.653367,0.814815,0.0,0.084307,0.534232,ESE,0.367188,E,ESE,...,0.418919,0.135417,0.07,0.519531,0.520871,0.559943,0.562783,0.752860,0.833333,No
144915,Uluru,0.640898,0.829218,0.0,0.084307,0.534232,E,0.382812,E,SE,...,0.297297,0.083333,0.07,0.478516,0.459165,0.559943,0.562783,0.784897,0.826923,No
144916,Uluru,0.703242,0.866255,0.0,0.084307,0.534232,E,0.406250,E,SSE,...,0.229730,0.083333,0.12,0.373047,0.370236,0.559943,0.125000,0.858124,0.905983,No


In [53]:
#OneHotEncoding
enc = OneHotEncoder(sparse_output=False, handle_unknown = 'ignore')
enc.fit(val_inputs[cat_cols])
#remember where we want to store the categorical data
enc_cols = enc.get_feature_names_out(cat_cols)
#
enc_data = enc.transform(val_inputs[cat_cols])
enc_data = pd.DataFrame(enc_data, columns= enc_cols)

In [72]:
fin_val = np.concat([val_inputs[num_cols], enc_data],axis =1)
final_cols = np.append(num_cols,enc_cols)
fin_val = pd.DataFrame(fin_val, columns= final_cols)
val_pred = model.predict(fin_val)

In [70]:
from sklearn.metrics import accuracy_score

In [73]:
accuracy_score(val_targets, val_pred)

0.8531804084498801

In [77]:
train_pred= model.predict(train_inputs)

In [78]:
accuracy_score(train_targets, train_pred)

0.8518900273502877